<h4>Global CO<sub>2</sub> Emissions Dataset from Maven Analytics</h4>

<p>Objectives:</p>
<ul>
    <li>What is the trend of CO<sub>2</sub> emissions in the world over time?
    Can it be related to the demographics?</li>
    <li>Which country contributes both the least and the most emissions?</li>
    <li>What are the major sources of these emissions? </li>
    <li>Is temperature related to the amount of CO<sub>2</sub> emissions?</li>
</ul>

In [20]:
#Libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [21]:
df = pd.read_csv("data/co2_data.csv")
df.shape

(50598, 79)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50598 entries, 0 to 50597
Data columns (total 79 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   country                                    50598 non-null  object 
 1   year                                       50598 non-null  int64  
 2   iso_code                                   42142 non-null  object 
 3   population                                 40008 non-null  float64
 4   gdp                                        14564 non-null  float64
 5   cement_co2                                 24974 non-null  float64
 6   cement_co2_per_capita                      22714 non-null  float64
 7   co2                                        31349 non-null  float64
 8   co2_growth_abs                             29010 non-null  float64
 9   co2_growth_prct                            25032 non-null  float64
 10  co2_including_luc     

<p>Filtering the data columns based on the objectives.</p>

In [23]:
#Columns containing demographics and CO2 emissions
df_demographics = df[['country', 'year', 'iso_code', 'population', 'gdp','co2',
'land_use_change_co2']]

# Columns containing the sources of emissions 
df_sources = df[['cement_co2','coal_co2','flaring_co2','gas_co2','land_use_change_co2',\
    'consumption_co2','oil_co2','other_industry_co2']]

# Columns containing temperature changes
df_temp = df[[col for col in df.columns if 'temperature' in col]]

#Columns for the methane and NOx
df_chem = df[['year','methane','nitrous_oxide']]

Table 1: Sources Table

In [24]:
# By pd.melt, reverse the columns into rows and maintain the values
df_sources =pd.melt(df_sources.reset_index(),id_vars=['index'],value_vars=df_sources.columns, \
    var_name="emission_sources",value_name="million_tons").copy()
    
df_sources = df_sources.groupby('emission_sources')['million_tons'].sum().reset_index()

In [25]:
# Adding the Percentage Column
df_sources['percentage'] = (df_sources['million_tons']/df_sources['million_tons'].sum())*100
df_sources['percentage'] = df_sources['percentage'].apply(lambda x: round(x,0) if x > 1 else round(x,2))
df_sources

,emission_sources,million_tons,percentage
0,cement_co2,209594.066,1.00
1,coal_co2,3870596.153,21.00
2,consumption_co2,6073187.268,33.00
3,flaring_co2,86909.154,0.47
4,gas_co2,1249162.183,7.00
5,land_use_change_co2,4333960.833,23.00
6,oil_co2,2789318.374,15.00
7,other_industry_co2,43974.095,0.24


Adding Foreign Key to Other Tables

In [26]:
# Combining those selected data columns
filtered_df = pd.concat([df_demographics,df_temp,df_chem],axis=1)
filtered_df.shape

(50598, 15)

In [27]:
# Add country foreign key to country
unique_countries = filtered_df['country'].unique()
country_values = pd.Series(np.arange(len(unique_countries)), index=unique_countries)
filtered_df['country_key'] = filtered_df['country'].apply(lambda x: str(100 + country_values[x]))

# Add Chemical Sources Key
filtered_df['chem_key'] = [int("20" + str(0 + x)) for x in range(len(filtered_df))]


# Add Temperature Change Key
filtered_df['temp_key'] = [int("30" + str(0+x)) for x in range(len(filtered_df))]


Key_columns = [col for col in filtered_df.columns if 'key' in col]
Key_columns


['country_key', 'chem_key', 'temp_key']

In [28]:
#Loop to drop and insert the key columns to the left side of the dataframe
for key in Key_columns:
    filtered_df[key] = filtered_df[key].astype(str)
    foreign_keys = filtered_df.pop(key)  
    filtered_df.insert(0, key, foreign_keys)

In [29]:
filtered_df.head()

,temp_key,chem_key,country_key,country,year,iso_code,population,gdp,co2,land_use_change_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,year,methane,nitrous_oxide
0,300,200,100,Afghanistan,1850,AFG,3752993.0,NaN,NaN,2.931,NaN,NaN,NaN,NaN,NaN,1850,NaN,NaN
1,301,201,100,Afghanistan,1851,AFG,3767956.0,NaN,NaN,2.968,0.165,0.0,0.0,0.0,0.0,1851,NaN,NaN
2,302,202,100,Afghanistan,1852,AFG,3783940.0,NaN,NaN,2.968,0.164,0.0,0.0,0.0,0.0,1852,NaN,NaN
3,303,203,100,Afghanistan,1853,AFG,3800954.0,NaN,NaN,3.004,0.164,0.0,0.0,0.0,0.0,1853,NaN,NaN
4,304,204,100,Afghanistan,1854,AFG,3818038.0,NaN,NaN,3.004,0.163,0.0,0.0,0.0,0.0,1854,NaN,NaN


<p>Summarizing Country Table, Chemical Composition Table, and Temperature Change Table</p>

In [30]:
country_df = filtered_df[['country_key','country','iso_code']]
chem_df = filtered_df[['chem_key','methane','nitrous_oxide']]
temp_df = filtered_df[['temp_key','share_of_temperature_change_from_ghg',\
    'temperature_change_from_ch4','temperature_change_from_co2','temperature_change_from_ghg', \
    'temperature_change_from_n2o']]



In [31]:
country_df = country_df['country'].unique().copy()
country_df.info()

AttributeError: 'numpy.ndarray' object has no attribute 'info'

In [ ]:
chem_df = chem_df.dropna(subset=['methane','nitrous_oxide']).copy()
chem_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6150 entries, 140 to 50595
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   chem_key       6150 non-null   object 
 1   methane        6150 non-null   float64
 2   nitrous_oxide  6150 non-null   float64
dtypes: float64(2), object(1)
memory usage: 192.2+ KB


In [ ]:
temp_df = temp_df.dropna(subset=['temperature_change_from_ch4',\
    'temperature_change_from_n2o']).copy()
    
temp_df['share_of_temperature_change_from_ghg']. \
    fillna(df['share_of_temperature_change_from_ghg'].mean())

temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37620 entries, 1 to 50597
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   temp_key                              37620 non-null  object 
 1   share_of_temperature_change_from_ghg  37620 non-null  float64
 2   temperature_change_from_ch4           37620 non-null  float64
 3   temperature_change_from_co2           37620 non-null  float64
 4   temperature_change_from_ghg           37620 non-null  float64
 5   temperature_change_from_n2o           37620 non-null  float64
dtypes: float64(5), object(1)
memory usage: 2.0+ MB


<p>Demographics Table</p>

In [ ]:
demographics_df = filtered_df.drop(['country','share_of_temperature_change_from_ghg', \
    'temperature_change_from_ch4','temperature_change_from_co2', 'temperature_change_from_ghg', \
    'temperature_change_from_n2o', 'year', 'methane', 'nitrous_oxide'],axis=1)

In [ ]:
# Drop NaN values from CO2 emission columns
demographics_df = demographics_df.dropna(subset=['co2','land_use_change_co2']).copy()

# Rename co2 column based on the data definition
demographics_df = demographics_df.rename(columns={"co2":"co2 without land_use"}).reset_index()

# Include land use change for calculations
demographics_df['co2'] = demographics_df['co2 without land_use'] + demographics_df['land_use_change_co2']
demographics_df = demographics_df.drop(['land_use_change_co2'],axis=1).copy()
demographics_df.head()


,index,temp_key,chem_key,country_key,iso_code,population,gdp,co2 without land_use,co2
0,99,3099,2099,100,AFG,7356890.0,NaN,0.015,8.039
1,100,30100,20100,100,AFG,7480464.0,9.421400e+09,0.084,8.584
2,101,30101,20101,100,AFG,7571542.0,9.692280e+09,0.092,9.142
3,102,30102,20102,100,AFG,7667534.0,1.001733e+10,0.092,9.948
4,103,30103,20103,100,AFG,7764549.0,1.063052e+10,0.106,10.915


In [ ]:
demographics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24218 entries, 0 to 24217
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 24218 non-null  int64  
 1   temp_key              24218 non-null  object 
 2   chem_key              24218 non-null  object 
 3   country_key           24218 non-null  object 
 4   iso_code              21294 non-null  object 
 5   population            22836 non-null  float64
 6   gdp                   13230 non-null  float64
 7   co2 without land_use  24218 non-null  float64
 8   co2                   24218 non-null  float64
dtypes: float64(4), int64(1), object(4)
memory usage: 1.7+ MB


<h3>Saving the processed dataset</h3>

In [ ]:
#Sources Table
# df_sources.to_csv("clean_data/emission_sources.csv",index=False)

# Country Table
# country_df.to_csv("clean_data/country.csv",index=False)

#  Chemical Composition Table
# chem_df.to_csv("clean_data/chemical.csv",index=False)

# Temperature Change Table
# temp_df.to_csv("clean_data/temperature_change.csv",index=False)

# Demographics Table
# demographics_df.to_csv("clean_data/demographics.csv",index=False)